In [20]:
#REFERENCES:
# https://towardsdatascience.com/extracting-text-from-scanned-pdf-using-pytesseract-open-cv-cd670ee38052
# https://www.codegrepper.com/code-examples/python/pytesseract+pdf+to+text
# https://www.google.com/search?q=python+pytesseract+pdf+to+text&oq=python+pdf+pytesse&aqs=edge.2.69i57j0i22i30l2.9015j0j1&sourceid=chrome&ie=UTF-8

#https://stackoverflow.com/questions/67516273/remove-header-and-footer-from-pdftotext-module-in-python
#https://www.anycodings.com/1questions/155089/remove-header-and-footer-from-pdftotext-module-in-python
#https://pyimagesearch.com/2015/02/09/removing-contours-image-using-python-opencv/
#https://pdf.wondershare.com.br/pdf-knowledge/pdf-to-png-python.html

In [21]:
# INSTALATIONS, if necessary:
#!pip install opencv-python   # cv2
#!pip install pytesseract
#!pip install pdf2image
#!pip install poppler-utils
#!apt-get install poppler-utils
#!apt-get update
#!sudo apt-get install libleptonica-dev tesseract-ocr python3-pil tesseract-ocr-por tesseract-ocr-eng tesseract-ocr-script-latn

Defaulting to user installation because normal site-packages is not writeable


In [25]:
#import opencv-python

# IMPORTS:
import cv2
import pytesseract
from PIL import Image
from pdf2image import convert_from_path

In [ ]:
# VARIABLES:
custom_config = r'--oem 3 --psm 11'
file_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/"   #file_path = "/content/"
output_folder="/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs"  #arq = file_path+"00002067920105010076-29-03-2016.pdf"
arq = "0080300-68.2008.5.01.0016.pdf"
path_arq = file_path+arq
path_arq

In [ ]:
# TESTS:

In [ ]:
pages = convert_from_path(arq, 500, output_folder=output_folder, first_page=1, last_page=1, fmt='jpg')

images = []

for i in range(len(pages)):
    page_name = arq+"_page_"+str(i)+".jpg"
    pages[i].save(page_name, "JPEG")
    images.append(page_name)


In [ ]:
for i in range(len(images)):
    text = pytesseract.image_to_string(images[i], lang='por')
    print(text)

In [ ]:
for i in range(len(images)):
    img = cv2.imread(images[i])
    
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#blur = cv2.GaussianBlur(gray, (9,9), 0)
#thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
#text = pytesseract.image_to_string(blur, lang="por")
#text.replace("\t","")
#text.replace("\n","")
#print(text)

In [ ]:
# WORKING:

In [44]:
%%writefile HelperFunctions/DataWrangling/criaCorpusWithOCR.py

# 1.Extração de texto dos pdf das sentenças baixas
#
# Cria Corpus (DatasetClassification.CSV) das Sentenças do TRT1
# Le os arquivos .txt e armazena-os em uma lista e depois persiste-o em disco no formato .csv.
#
import io
import pathlib
import pandas as pd
import datasets
from datasets import load_dataset
from datasets import Dataset
import os
import shutil
from os import listdir
from os.path import isfile, join
import re
import string
# IMPORTS:
import cv2
import pytesseract
from PIL import Image, ImageOps
from pdf2image import convert_from_path
import numpy

    
class Corpus():
    
    def __init__(self):
        # Agrupar estas variaveis em um arquivo para carregar nas classes. VARIABLES:
        self.custom_config = r'--oem 1 --psm 4'
        self.pdfs_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/"   #file_path = "/content/"
        self.file_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFsProcessing/"
        self.output_folder="/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/"  #arq = file_path+"00002067920105010076-29-03-2016.pdf"
        #self.arq = "0080300-68.2008.5.01.0016.pdf"
        self.arq = "" #"0012600-80.2005.5.01.0016.pdf"
        self.path_arq = "" #self.file_path+self.arq
        self.imgNames = []
        self.textos = []
        self.pages = []
        self.imgList = []
        


            
    def remove_jpg(self):
        jpgList = os.listdir(self.output_folder)      
        for imgFile in jpgList:
            if imgFile.endswith('.jpg'):
                os.remove(self.output_folder+imgFile)
                
    def remove_sub_jpg(self):
        img_sub_List = os.listdir(self.output_folder+"imgs/")  
        for k in img_sub_List:
            if k.endswith('.jpg'):
                os.remove(self.output_folder+"imgs/"+k)
      
        

    
    def enhance_img(self):
        print("Tam. imgList", len(self.imgList))
        for i, imgPage in enumerate(self.imgList):
                        
            # open method used to open different extension image file
            # pil_image = Image.open(self.output_folder+self.arq+"_page_"+str(i)+".jpg") 
            #width, height = pil_image.size
            # gray_img    = ImageOps.grayscale(pil_image)
            # bk_wh_img   = gray_img.convert('1', dither=Image.NONE) # convert image to black and white
            print("imgList: ", self.imgList[i])
            
            img = cv2.imread(imgPage, cv2.IMREAD_GRAYSCALE)
            # You may need to convert the color.
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            (thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            #dst = cv2.detailEnhance(im_bw, sigma_s=10, sigma_r=0.15)
            
            '''
            pil_image = Image.fromarray(im_bw)

            frac   = 0.89
            left   = pil_image.size[0]*((1-frac)/2)
            upper  = pil_image.size[1] # pil_image.size[1]*((1-frac)/2)
            right  = pil_image.size[0]-((1-frac)/2)*pil_image.size[0]
            bottom = pil_image.size[1]-((1-frac)/2)*pil_image.size[1]
        
            cropped_img = pil_image.crop((left*1, upper*1, right, bottom*2.5))
            #page_name = self.output_folder+self.arq+"_page_"+str(i)+".jpg"
            cropped_img.save(imgPage,"JPEG")  # Ou .save(page_name, "JPEG")    
            '''
            
            cv2.imwrite(imgPage, im_bw)
        
        
    def pdftoimg(self):
        
        self.pages = convert_from_path(self.path_arq, 500, output_folder=self.output_folder+"imgs/", fmt='jpg') # first_page=1, last_page=1, 

        for i, page in enumerate(self.pages):
            page_name = self.output_folder+self.arq+"_page_"+str(i)+".jpg"
            page.save(page_name, "JPEG")
            self.imgList.append(page_name)
            print("Pagina: ", page_name)
            
        self.enhance_img()
        self.remove_sub_jpg()
        
        # Lista com endereço dos arquivos jpg do pdf a serem processados pelo OCR.
        #self.imgList = os.listdir(self.output_folder)
        #self.imgList = [self.output_folder+t for t in self.imgList if t.endswith('.jpg')]
        #self.imgList.sort(key=lambda fname: int(fname.split('.')[0]))  # Ordena
        #for i in range(len(self.imgList)):
        #    print("conteudo de imgList: ", self.imgList[i])
                     
    

    def process_text(self, s_text):

        #s_text.replace("\n", " ")
        #s_text.replace('-\n', ' ')

        s_text = s_text.replace(" ","<==>") # Como o filtro de caracteres imprimiveis estava removendo os espaços
        
        printable = set(string.printable)
        printable.remove('\x0b')
        printable.remove('\x0c')
        #printable.remove('\t')
        #printable.remove('\n')
        spec_char = {'ã','ẽ','õ','á','é','í','ó','ú','â','ê','î','ô','û','Ã','Ẽ','Õ','Á','É','Í','Ó','Ú','Â','Ê','Î','Ô','Û','ü','Ü','ç','Ç','º','ª','§'}
        printable = printable.union(spec_char)
        
        s_text = "".join(filter(lambda x: x in printable, s_text))  # ou s=re.sub(r'[\W_]+', '', res) #  Filtra caracteres alfanumericos
        s_text = s_text.replace("<==>"," ") # Restitui os espaços para a string.
        #print("Na saída da funcao imgtotext: ",s_text) 
        
        return s_text

        
        
    
    def imgtotext(self):
        
        self.textos = []
        #imgList = os.listdir(self.output_folder)
        
        for imgFile in self.imgList:
            if imgFile.endswith('.jpg'):
                # Recognize the text as string in image using pytesserct 
                #pytesseract.pytesseract.tesseract_cmd = tesseract_path
                #text = str(((pytesseract.image_to_string(Image.open(img))))) 
                text = str(((pytesseract.image_to_string(imgFile, lang='por', config=self.custom_config)))) 
                self.textos.append(text)
                #print("DENTRO DO LOOP de extração dos textos das paginas", text)
        
        
        res = " ".join(self.textos) # Transforma uma lista de strings (para cada pagina) em uma string longa.
        #print("Apos concatenar a lista de paginas, antes do processamento de string: ", res)
        
        # Realiza manipulacoes de limpeza do texto
        res = self.process_text(res)  
        #print("Apos concatenar a lista de paginas, antes do processamento de string: ", res)
        
        # Remove temporary IMGs/*.jpg files for OCR:
        self.remove_jpg()
        self.imgList = []  #Ultima alteração
        #print(len(self.images))  # imprime a quantidade de paginas.
       
        return res

    
    
    def proccess_folder(self):
        
        txt_content = ""
        #dataFile = {}
        directoryList = os.listdir(self.file_path)
        #self.remove_jpg()
        
        for pdfFile in directoryList:
            
            if pdfFile.endswith('.pdf'):
                self.arq = pdfFile
                self.path_arq = self.file_path+self.arq
                #self.pages = []
                self.pdftoimg()
                txt_content = self.imgtotext()  #
                #print("No loop de proccess_folder que chama imgtotext",txt_content)   #
                with open(self.file_path+self.arq+'.txt', 'w') as f:
                    f.write(txt_content)
                f.close()
                
            txt_content = ""

                
                
    def reload_files(self):
        
        # 
        directoryList = os.listdir(self.pdfs_path)
        batch_step = 100
        for i in range(0,len(directoryList), batch_step):
            
            for j in range(i,i+batch_step):
                print(i,j,"moving for preparing proccess: ", directoryList[j])
                if directoryList[j].endswith('.pdf'):
                    shutil.move(self.pdfs_path+directoryList[j], self.file_path)
                    
            self.proccess_folder()
            
            for j in range(i,i+batch_step):
                print(i,j,"moving processed files: ", directoryList[j])
                shutil.move(self.file_path+directoryList[j],self.pdfs_path+"Processed/")
                shutil.move(self.file_path+directoryList[j]+".txt",self.file_path+"txts/")
           
            '''
            self.imgList = os.listdir(self.output_folder)
            for k in self.imgList:
                if k.endswith('.jpg'):
                    os.remove(self.output_folder+k)
            '''
            
        print("PDF files processed in: ", self.pdfs_path+"Processed/")
        print("TXT files generated in: ", self.file_path+"txts/")
        print("Temporary images files used in: ", self.output_folder)
        print("Eventually unprocessed/error files in: ", self.pdfs_path)

            
            

Overwriting HelperFunctions/DataWrangling/criaCorpusWithOCR.py


In [45]:
# 2. Criação do objeto
from HelperFunctions.DataWrangling.criaCorpusWithOCR import Corpus
%run -i HelperFunctions/DataWrangling/criaCorpusWithOCR.py
corpus = Corpus()

In [46]:
%%time
corpus.reload_files()

0 0 moving for preparing proccess:  01028309720175010421-DEJT-06-03-2018.pdf
0 1 moving for preparing proccess:  01002131620175010050-DEJT-14-09-2017.pdf
0 2 moving for preparing proccess:  01003933820175010048-DEJT-22-11-2017.pdf
0 3 moving for preparing proccess:  01018231920175010244-DEJT-19-09-2018.pdf
0 4 moving for preparing proccess:  01004818920165010055-DOERJ-29-11-2017.pdf
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01028309720175010421-DEJT-06-03-2018.pdf_page_0.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01028309720175010421-DEJT-06-03-2018.pdf_page_1.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01028309720175010421-DEJT-06-03-2018.pdf_page_2.jpg
Tam. imgList 3
imgList:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01028309720175010421-DEJT-06-03-2018.pdf_page_0.jpg
imgList:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01028309720175010421-DEJT-06-03-2018.pdf_page_1.jpg
imgList:  /home/info/MyNotebooks/Data

Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_8.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_9.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_10.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_11.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_12.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_13.jpg
Tam. imgList 14
imgList:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_0.jpg
imgList:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-DEJT-19-07-2018.pdf_page_1.jpg
imgList:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01017784720175010201-

5 5 moving processed files:  01017784720175010201-DEJT-19-07-2018.pdf
5 6 moving processed files:  01011303320175010471-DEJT-01-09-2017.pdf
5 7 moving processed files:  01014738520165010205-DEJT-17-02-2017.pdf
5 8 moving processed files:  01012354420185010028-DEJT-06-11-2019.pdf
5 9 moving processed files:  01009744720165010029-DOERJ-11-04-2017.pdf
10 10 moving for preparing proccess:  01025861820175010471-DEJT-31-10-2018.pdf
10 11 moving for preparing proccess:  00106987520155010070-DEJT-28-03-2017.pdf
10 12 moving for preparing proccess:  01014996120175010201-DEJT-01-06-2018.pdf
10 13 moving for preparing proccess:  01006945120175010511-DEJT-28-07-2017.pdf
10 14 moving for preparing proccess:  01019795420175010002-DEJT-10-05-2019.pdf
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01025861820175010471-DEJT-31-10-2018.pdf_page_0.jpg
Pagina:  /home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/01025861820175010471-DEJT-31-10-2018.pdf_page_1.jpg
Pagina:  /home/info/MyNotebook

KeyboardInterrupt: 

In [ ]:
s = "some\x00string. with\x15 funny characters"

In [ ]:
import string

space_char = set(" ")
prtle = set(string.printable)
printable = space_char.union(prtle)
#filter(lambda x: x in printable, s)
print(len(printable))

In [ ]:
print(space_char)

In [ ]:
print(printable)

In [ ]:
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
type(printable)

In [ ]:
set1 = set(string.printable)
set2 = set(" ")

set3 = set1 | set2
 
# union of two sets
print("set1 U set2 : ", set3)

In [ ]:
print(len(set1), ' ', len(set3))

In [ ]:
spec_char = {'ã','ẽ','õ','á','é','í','ó','ú','â','ê','î','ô','û','Ã','Ẽ','Õ','Á','É','Í','Ó','Ú','Â','Ê','Î','Ô','Û','ü','Ü','ç','Ç','º','ª','§'}
printable = printable.union(spec_char)
print(len(printable))

In [ ]:
type(printable)

In [ ]:
print(len(printable), ' ', len(set3))

In [ ]:
print("ç" in printable)

In [ ]:
set1 = {2, 4, 5, 6}
set2 = {4, 6, 7, 8}
set3 = {7, 8, 9, 10}
 
# union of two sets
print("set1 U set2 : ", set1.union(set2))

In [ ]:
printable.remove('\x0b')

In [ ]:
print(printable)

In [ ]:
caract = ['\x0b', '\x0c', '\t', '\n']

In [ ]:
print(lambda x: x in caract)

In [ ]:
printable.remove(lambda x: x in caract)

In [ ]:
%%time
corpus.proccess_folder()

In [ ]:
%%time
corpus.pdftoimg()

In [ ]:
%%time
corpus.crop_img()

In [ ]:
%%time
print(corpus.imgtotext())
# psm 11

In [ ]:
%%time
print(corpus.imgtotext())
# psm 11

In [ ]:
%%time
print(corpus.imgtotext())
# psm 11

In [ ]:
%%time
print(corpus.imgtotext())
# psm 11

In [ ]:
%%time
print(corpus.imgtotext())
# psm 4

In [ ]:
%%time
print(corpus.imgtotext())
# --oem 1 --psm 4

In [ ]:
%%time
print(corpus.imgtotext())
# psm


In [ ]:
%%time
print(corpus.imgtotext())
# psm

In [ ]:
#!pip install ocrmypdf

In [ ]:
import ocrmypdf
file = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/0080300-68.2008.5.01.0016.pdf"
result = ocrmypdf.ocr(file, 'OCR_'+file, output_type='pdf', skip_text=True, deskew=True)

In [ ]:
!pip install PyMuPDF

In [ ]:
import fitz

In [ ]:
file = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/0080300-68.2008.5.01.0016.pdf"
doc = fitz.open(file)

for page_num in range(doc.pageCount):
        page = doc.loadPage(page_num)
        pages_df = pages_df.append({'text': page.getText('text')}, ignore_index=True)
        
        
extraction_pdfs[file] = pages_df  

In [ ]:
for file in file_list:
    try:
        result = ocrmypdf.ocr(file, 'OCR_'+file,output_type='pdf',skip_text=True,deskew=True)
    except Exception as e:
        if hasattr(e,'message'):
            error_log[file] = e.message
        else:
            error_log[file] = e
        continue
        
'''

In [ ]:
# https://www.anycodings.com/1questions/155089/remove-header-and-footer-from-pdftotext-module-in-python